In [148]:
import pandas as pd
import numpy as np
import requests
import re
from bs4 import BeautifulSoup
import json
from datetime import datetime, date, time, timedelta


In [149]:
listaJson = []
print(listaJson)
options = Options()
options.binary_location = r'C:\Program Files\Mozilla Firefox\firefox.exe'
#options.headless = True
options.add_argument("--disable-notifications")
options.add_argument("--mute-audio")
#driver = webdriver.Firefox(options=options, executable_path = 'C:\python-geckodriver\geckodriver.exe')

[]


In [150]:
def definirParams(auth: "", path: "", referer: ""):
    PARAMS = {
            "authority": auth,
            "method": "GET",
            "path":path,
            "scheme": "https",
            "referer": referer,
            "sec-fetch-mode": "navigate",
            "sec-fetch-site": "same-origin",
            "sec-fetch-user": "?1",
            "upgrade-insecure-requests": "1",
            "user-agent": 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.130 Safari/537.36',
            }
    return PARAMS

def criarJson(url, titulo, bairro, preco, dia_postagem, descricao,metragem):
    print("titulo: " + str(titulo))
    print("Preco: " + str(preco))
    print(bairro)
    print("----")
        
    json = { 
    "preco": preco,
    "titulo": titulo,
    "bairro": bairro,
    "dia_postagem" : dia_postagem,
    "url": url,
    "descricao": descricao,
    "metragem": metragem}
    
    #adiciona na lista GERAL de JSON
    listaJson.append(json)

def retornarSoupSimples(url):
    auth = url.split("/")[6]
    path = url.split("/")[7]
    referer = url
    PARAMS = definirParams(auth = auth,path = path, referer = url )
    page = requests.get(url = url, headers = PARAMS)
    soup = BeautifulSoup(page.content,'lxml')
    return soup

def retornarSoupSimples2(url):
    auth = url.split("/")[2]
    path = url.split("/")[3]
    referer = url
    PARAMS = definirParams(auth = auth,path = path, referer = url )
    page = requests.get(url = url, headers = PARAMS)
    soup2 = BeautifulSoup(page.content,'lxml')
    return soup2

def buscarDadosOLX(pages = 50):
        
    for x in range(1, pages):
        print(" LOOP NÚMERO: " + str(x))
        url = "https://www.olx.com.br/imoveis/venda/casas/estado-mg/regiao-de-uberlandia-e-uberaba/triangulo-mineiro/uberlandia?o="+str(x)
        soup = retornarSoupSimples(url)
        results = soup.find_all("li",{"class":"sc-1mburcf-1"})
        itens = results
        print(len(itens))
                
        for item in itens:
            url = item.findAll("a")[0]["href"]
            titulo = item.find("h2").text
            
            try:
                preco = item.find_all("span",{"class":"main-price"})[0].text
                preco = preco.split("R$")[1]
                preco = float(preco.replace(".", ""))
            except:
                preco = None
            
            localizacao = item.find_all("span", text=re.compile("Uberlândia"))[0].text
            try:
                bairro = localizacao.split(",")[1]
            except:
                bairro=localizacao
            
            try:  
                dataPostagem = item.findAll("span", class_="adDate")[1].text
                
            except:
                dataPostagem = item.findAll("span", class_="adDate")[0].text
            
            dia_postagem = dataPostagem.split(",")[0]
            if dia_postagem =="Hoje":
                dia_postagem = date.today()
            elif dia_postagem == "Ontem":
                dia_postagem = date.today()-timedelta(days=1)
            else:
                dia_postagem = dia_postagem
            
            url2 = retornarSoupSimples2(url)
            descricao = url2.find_all(attrs={"data-section": "description"})[0].text
            
            try:
                metragem= item.find_all(attrs={"aria-label":re.compile("metros")})[0].text
                metragem= metragem.split("m")[0]
            except:
                metragem = None
                          
            
            criarJson(url, titulo, bairro, preco, dia_postagem, descricao, metragem)
            

In [151]:
buscarDadosOLX(pages=3)

 LOOP NÚMERO: 1
50
titulo: Casa para venda tem 240 metros quadrados com 3 quartos em Morada da Colina - Uberlândia - 
Preco: 1100000.0
 Morada da Colina
----
titulo: Casa Rua Comercial, próximo Lotérica e supermercado
Preco: 380000.0
 São Jorge
----
titulo: Casa em condomínio zona sul
Preco: 1850000.0
 Jardim Karaíba
----
titulo: Venda Casa Loteamento São Bento
Preco: 252000.0
 Jardim Canaã
----
titulo: Casa no condomínio Reserva dos Ipês com três quartos.
Preco: 1400000.0
 Novo Mundo
----
titulo: Casa para venda tem 78 metros quadrados com 2 quartos em Mansour - Uberlândia - MG
Preco: 370000.0
 Mansour
----
titulo: Casa de condomínio térrea para venda tem 236 metros quadrados com 3 quartos
Preco: 1300000.0
 Lagoinha
----
titulo: Bairro Ipanema - Casa 2/4 com Suite - Proximo a Ubs - 3 casas no lote - 1617udi
Preco: 220000.0
 Jardim Ipanema
----
titulo: Sobrado com 3 quartos para venda na Zona ul
Preco: 400000.0
 Jardim Karaíba
----
titulo: CASA EM CONDOMÍNIO RESIDENCIAL em UBERLÂNDIA -

titulo: Casa de condomínio térrea para venda tem 200 metros quadrados com 3 quartos
Preco: 1850000.0
 Jardim Inconfidência
----
titulo: Casa à venda 3 Quartos, 1 Suite, 2 Vagas, 82M², Vida Nova, Uberlândia - MG
Preco: 395000.0
 Novo Mundo
----
titulo: UBERLÂNDIA - Casa de Condomínio - Alto Umuarama
Preco: 1200000.0
 Brasil
----
titulo: Sobrado à venda 5 Quartos, 4 Suites, 5 Vagas, 525M², Morada da Colina, Uberlândia - MG
Preco: 1500000.0
 Morada da Colina
----
titulo: Casa para venda possui 195 metros, Área gourmet c/ Spa, em Jardim Americana, Uberlândia MG
Preco: 850000.0
 Jardim América I
----
titulo: Casa com 2/4   no Bairro São Jorge, 90m, churrasqueira, garagem 4 vagas,  Sala ampla tv/ja
Preco: 198000.0
 São Jorge
----
titulo: Casa no condomínio varanda sul 
Preco: 1550000.0
 Mansões Aeroporto
----
titulo: Casa Usada Conservada  no Planalto! Localização Excelente! Confira!
Preco: 350000.0
 Planalto
----
titulo: Casa térrea com 3 suítes, escritório e piscina em condomínio na região

In [113]:
df = pd.DataFrame(listaJson)


In [103]:
df.head(10)

,preco,titulo,bairro,dia_postagem,url,descricao,metragem
0,600000.0,"Casa com 4 dormitórios à venda, 170 m² por R$ ...",Nossa Senhora Aparecida,2023-03-02,https://mg.olx.com.br/regiao-de-uberlandia-e-u...,DescriçãoCódigo do anúncio: CA1072\n\nCasa de ...,170
1,300000.0,"Casa à venda, 3 quartos, 1 vaga, Martins - Ube...",Martins,2023-03-02,https://mg.olx.com.br/regiao-de-uberlandia-e-u...,DescriçãoCódigo do anúncio: 14488\n\nCasa com ...,120
2,2520000.0,UBERLÂNDIA - Casa de Condomínio - Condomínio G...,Nova Uberlândia,2023-03-01,https://mg.olx.com.br/regiao-de-uberlandia-e-u...,DescriçãoCódigo do anúncio: CA00054\n\nCasa Du...,313
3,1750000.0,Venda Casa CONDOMINIO SPLENDIDO,Jardim Botânico,2023-03-01,https://mg.olx.com.br/regiao-de-uberlandia-e-u...,DescriçãoCódigo do anúncio: 12577\n\nGARAGEM 0...,201
4,250000.0,Casa Bairro Pampulha Excelente Localização,Pampulha,2023-03-01,https://mg.olx.com.br/regiao-de-uberlandia-e-u...,DescriçãoCódigo do anúncio: 1017509613\n\nOBSE...,300
5,270000.0,Venda Casa PLANALTO,Planalto,2023-03-01,https://mg.olx.com.br/regiao-de-uberlandia-e-u...,DescriçãoCódigo do anúncio: 21552\n\nCasa loca...,130


In [114]:
df_filtro_preco = df[df["preco"].between(950000, 1400000, inclusive="both")]
df_filtro_preco.head()

,preco,titulo,bairro,dia_postagem,url,descricao,metragem
0,1000000.0,Casa à venda no Bairro Lídice em Uberlândia,Lídice,2023-03-02,https://mg.olx.com.br/regiao-de-uberlandia-e-u...,DescriçãoCódigo do anúncio: CA1407\n\n* Casa t...,200
6,1200000.0,Sobrado para venda tem 232 metros quadrados co...,Granada,2023-03-02,https://mg.olx.com.br/regiao-de-uberlandia-e-u...,DescriçãoCódigo do anúncio: 9KF9UP\n\nSobrado ...,Sobrado para venda te
13,1350000.0,Casa à venda no Condomínio fechado Reserva Dos...,Novo Mundo,2023-03-02,https://mg.olx.com.br/regiao-de-uberlandia-e-u...,DescriçãoCódigo do anúncio: CA00502\n\nCasa à ...,225
14,1200000.0,Casa à venda no bairro Tocantins em Uberlândia,Tocantins,2023-03-02,https://mg.olx.com.br/regiao-de-uberlandia-e-u...,DescriçãoCódigo do anúncio: CA3887\n\n*Ótimo s...,398
23,1350000.0,"Condomínio Paradiso Ecológico, Casa 3/4, Sala ...",Bosque dos Buritis,2023-03-02,https://mg.olx.com.br/regiao-de-uberlandia-e-u...,DescriçãoCódigo do anúncio: 1747-A\n\n-Casa no...,203


## Executar apenas para conhecer o dataframe filtrado pelo preço, verificar os bairros

In [122]:
with pd.ExcelWriter("DFfiltropreco.xlsx", options={'strings_to_urls': False}) as writer:
    df_filtro_preco.to_excel(writer,"DFfiltropreco.xlsx")

C:\Users\guilh\AppData\Local\Temp\ipykernel_24636\3133342000.py:1: FutureWarning: Use of **kwargs is deprecated, use engine_kwargs instead.
  with pd.ExcelWriter("DFfiltropreco.xlsx", options={'strings_to_urls': False}) as writer:


In [141]:
lista_bairros = [" Altamira", " Cidade Jardim", " City Uberlândia", " Gávea", " Itapema Sul", " Jardim Botnico", 
                            " Jardim Colina", " Jardim Inconfidncia", " Jardim Indaiá", " Jardim Karaíba", " Jardim Sul", 
                            " Laranjeiras",  " Lídice", " Morada da Colina", " Nova Uberlândia", " Vigilato Pereira"]
df_filt_pb = df_filtro_preco[df_filtro_preco["bairro"].isin(lista_bairros)]
df_filt_pb


,preco,titulo,bairro,dia_postagem,url,descricao,metragem
0,1000000.0,Casa à venda no Bairro Lídice em Uberlândia,Lídice,2023-03-02,https://mg.olx.com.br/regiao-de-uberlandia-e-u...,DescriçãoCódigo do anúncio: CA1407\n\n* Casa t...,200
28,1400000.0,Casa à venda no bairro Cidade Jardim em Uberlâ...,Cidade Jardim,2023-03-02,https://mg.olx.com.br/regiao-de-uberlandia-e-u...,DescriçãoCódigo do anúncio: CA3885\n\n*Maravil...,230
144,1300000.0,UBERLANDIA - Casa Padrão - Jardim Karaíba,Jardim Karaíba,2023-03-01,https://mg.olx.com.br/regiao-de-uberlandia-e-u...,DescriçãoCódigo do anúncio: 6728\n\nExcelente ...,209
145,1200000.0,Excelente casa cond Village Karaiba,Jardim Karaíba,2023-03-01,https://mg.olx.com.br/regiao-de-uberlandia-e-u...,DescriçãoCódigo do anúncio: Casa cond Village ...,194
146,1150000.0,UBERLÂNDIA - Casa Padrão - JARDIM KARAÍBA,Jardim Karaíba,2023-03-01,https://mg.olx.com.br/regiao-de-uberlandia-e-u...,DescriçãoCódigo do anúncio: CA00067\n\nExcelen...,342
...,...,...,...,...,...,...,...
2452,970000.0,Execelente casa com 230 m² na Zona Sul,Jardim Sul,24 de fev,https://mg.olx.com.br/regiao-de-uberlandia-e-u...,DescriçãoCódigo do anúncio: 8HNP3O\n\n- Casa n...,230
2475,1100000.0,excelente localização no jardim karaiba,Jardim Karaíba,24 de fev,https://mg.olx.com.br/regiao-de-uberlandia-e-u...,DescriçãoCódigo do anúncio: casa jd Karaiba\n\...,200
2478,1300000.0,Venda Casa JARDIM KARAIBA,Jardim Karaíba,24 de fev,https://mg.olx.com.br/regiao-de-uberlandia-e-u...,DescriçãoCódigo do anúncio: 15677\n\nSala home...,209
2484,1250000.0,Casa para venda 3 SUÍTES cidade jardim uberlân...,Cidade Jardim,24 de fev,https://mg.olx.com.br/regiao-de-uberlandia-e-u...,DescriçãoCódigo do anúncio: CA1978\n\nSala de ...,177


## Executar apenas para conhecer o dataframe filtrado pelo preço e bairros

In [142]:
with pd.ExcelWriter("DFfiltroPB.xlsx", options={'strings_to_urls': False}) as writer:
    df_filt_pb.to_excel(writer,"DFfiltroPB.xlsx")

C:\Users\guilh\AppData\Local\Temp\ipykernel_24636\3740428937.py:1: FutureWarning: Use of **kwargs is deprecated, use engine_kwargs instead.
  with pd.ExcelWriter("DFfiltroPB.xlsx", options={'strings_to_urls': False}) as writer:


In [147]:
df_filt_pb[df_filt_pb['descricao'].str.contains("escrit")]

,preco,titulo,bairro,dia_postagem,url,descricao,metragem
269,1370000.0,"Casa para venda com 3 suítes, bairro Jardim Ka...",Jardim Karaíba,2023-03-01,https://mg.olx.com.br/regiao-de-uberlandia-e-u...,DescriçãoCódigo do anúncio: RA5208... ver núme...,213
272,950000.0,"Casa à venda, 3 quartos, 1 suíte, 2 vagas, Cid...",Cidade Jardim,2023-03-01,https://mg.olx.com.br/regiao-de-uberlandia-e-u...,DescriçãoCódigo do anúncio: 68863\n\nÓtimo imó...,192
315,1370000.0,Casa térrea 3 suítes Jardim Karaíba,Jardim Karaíba,2023-03-01,https://mg.olx.com.br/regiao-de-uberlandia-e-u...,DescriçãoCódigo do anúncio: 529APM\n\nSão 3 su...,213
339,1100000.0,Linda casa sobrado.,Cidade Jardim,2023-03-01,https://mg.olx.com.br/regiao-de-uberlandia-e-u...,DescriçãoCódigo do anúncio: M2998\n\n. salas e...,280
397,1400000.0,CASA A VENDA BAIRRO VIGILATO PEREIRA PERTO DO ...,Vigilato Pereira,2023-03-01,https://mg.olx.com.br/regiao-de-uberlandia-e-u...,DescriçãoCódigo do anúncio: CA00105\n\nCasa a ...,269
516,1200000.0,Linda casa estilo sobrado no bairro Cidade Jar...,Cidade Jardim,2023-03-01,https://mg.olx.com.br/regiao-de-uberlandia-e-u...,DescriçãoCódigo do anúncio: 24961\n\nLinda cas...,319
588,1029400.0,"Casa com 3 quartos à venda, 174 m² à partir de...",City Uberlândia,28 de fev,https://mg.olx.com.br/regiao-de-uberlandia-e-u...,DescriçãoCódigo do anúncio: CA1844\n\nCasa com...,175
602,1300000.0,CASA TÉRREA NOVA A VENDA EM UBERLÂNDIA BAIRRO...,Jardim Karaíba,28 de fev,https://mg.olx.com.br/regiao-de-uberlandia-e-u...,DescriçãoCódigo do anúncio: CA00128\n\nCasa té...,209
654,1300000.0,Casa com 5 suítes para venda no bairro Morada ...,Jardim Colina,28 de fev,https://mg.olx.com.br/regiao-de-uberlandia-e-u...,DescriçãoCódigo do anúncio: CA00807\n\nCasa co...,439
661,1370000.0,Térrea para venda tem 205 metros quadrados com...,Jardim Karaíba,28 de fev,https://mg.olx.com.br/regiao-de-uberlandia-e-u...,DescriçãoCódigo do anúncio: CA3082\n\n* Linda ...,Térrea para venda te


In [21]:
with pd.ExcelWriter("CasasOLX.xlsx", options={'strings_to_urls': False}) as writer:
    df.to_excel(writer,"CasasOLX.xlsx")

C:\Users\guilh\AppData\Local\Temp\ipykernel_18176\1667572390.py:1: FutureWarning: Use of **kwargs is deprecated, use engine_kwargs instead.
  with pd.ExcelWriter("CasasOLX.xlsx", options={'strings_to_urls': False}) as writer:
